NEW CODE PRACTICE

In [3]:
import pandas as pd

#Load Beneficiaries (2008-2010)

beneficiaries = []

for year in [2008, 2009, 2010]:
    filename = f"/content/drive/MyDrive/GOOGLE COLAB/Dataset/New_Dataset_1/DE1_0_{year}_Beneficiary_Summary_File_Sample_20.csv"
    df = pd.read_csv(filename,dtype=str)
    print(filename)
    df['Year']=year
    beneficiaries.append(df)
beneficiaries = pd.concat(beneficiaries, ignore_index=True)
beneficiaries.shape

/content/drive/MyDrive/GOOGLE COLAB/Dataset/New_Dataset_1/DE1_0_2008_Beneficiary_Summary_File_Sample_20.csv
/content/drive/MyDrive/GOOGLE COLAB/Dataset/New_Dataset_1/DE1_0_2009_Beneficiary_Summary_File_Sample_20.csv
/content/drive/MyDrive/GOOGLE COLAB/Dataset/New_Dataset_1/DE1_0_2010_Beneficiary_Summary_File_Sample_20.csv


(343827, 33)

In [4]:
#Load Carrier Files

df_carrier = pd.read_csv("/content/drive/MyDrive/GOOGLE COLAB/Dataset/New_Dataset_1/DE1_0_2008_to_2010_Carrier_Claims_Sample_20A_1 - Copy-csv.csv", dtype=str)
df_carrier = df_carrier.sample(n=343827, random_state=42)
df_carrier.shape

(343827, 142)

In [14]:
#Merging beneficary dataset
df_carrier['Year'] = pd.to_datetime(df_carrier['CLM_FROM_DT'], errors='coerce').dt.year
df = df_carrier.merge(beneficiaries, on=['DESYNPUF_ID'], how='left')
print("Merged dataset:", df.shape)

Merged dataset: (1026402, 179)


In [8]:
#Calculating age
beneficiaries['Bene_Age'] = pd.to_numeric(beneficiaries['Year']) - pd.to_numeric(beneficiaries['BENE_BIRTH_DT'].str[:4],errors='coerce')

In [9]:
# Sex 1=Male and 2=Female
beneficiaries['Sex'] = beneficiaries['BENE_SEX_IDENT_CD'].map({'1': 'M', '2': 'F'})

In [10]:
# Counting the chronics counts
chronic_cols = [
    'SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD',
    'SP_DEPRESSN', 'SP_DIABETES', 'SP_ISCHMCHT', 'SP_OSTEOPRS',
    'SP_RA_OA', 'SP_STRKETIA'
]

# Count how many chronic conditions = 1
beneficiaries['CHRONIC_CNT'] = beneficiaries[chronic_cols].apply(lambda x: (x == '1').sum(), axis=1)


In [11]:
# Total Coverage months
beneficiaries['TOTAL_COVERAGE_MONS'] = (
    beneficiaries['BENE_HI_CVRAGE_TOT_MONS'].astype(float) +
    beneficiaries['BENE_SMI_CVRAGE_TOT_MONS'].astype(float) +
    beneficiaries['BENE_HMO_CVRAGE_TOT_MONS'].astype(float) +
    beneficiaries['PLAN_CVRG_MOS_NUM'].astype(float)
)

In [20]:
#FEATURE ENGINEERING FOR CARRIER DATASET
# Only one claims dataset has been used so no merge required.

# Duration Calculation
df_carrier['SERVICE_DURATION']= pd.to_numeric(df_carrier['CLM_THRU_DT'])-pd.to_numeric(df_carrier['CLM_FROM_DT'])
#Output will be in the format of days

df_carrier[['CLM_FROM_DT','CLM_THRU_DT','SERVICE_DURATION']]

,CLM_FROM_DT,CLM_THRU_DT,SERVICE_DURATION
781974,20100329,20100329,0
937737,20101207,20101207,0
907828,20100721,20100721,0
784628,20081125,20081125,0
662460,20090127,20090127,0
...,...,...,...
135634,20081008,20081008,0
418581,20090404,20090404,0
78384,20090209,20090209,0
988273,20081107,20081107,0
